In [4]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [5]:
def MIP_IM():
    attributes = ['age']#, 'gender', 'age', 'region', 'ethnicity']
  
    budget = 25
    m = 100
    for attribute in attributes:
        for index in range(20):
            with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
                main_graph = pickle.load(f)

                samples = []
                for j in range(m):
                    G = nx.DiGraph()
                    for u in main_graph.nodes():
                        G.add_node(u)
                    for u,v in main_graph.edges():
                        if main_graph[u][v]['p']> random.random():
                            G.add_edge(u, v)
                    samples.append(G)

                labels = nx.get_node_attributes(main_graph, attribute)
                label_dict = {}
                for i in range(len(main_graph.nodes())):
                    label = labels[i].encode('utf-8')
                    if label in label_dict.keys():
                        label_dict[label].append(i)
                    else:
                        label_dict[label] = [i] 
                        
                model = Model('mean_'+str(attribute)+'_'+str(index))
                min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)


                mvars = []
                #active nodes
                avars = []
                #seed nodes
                svars = []
                var_seed_dict = {}
                var_active_dict = {}
                var_mean_dict = {}

                for j in range(len(main_graph.nodes())):
                    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    svars.append(s)
                    var_seed_dict[j] = s

                for sample_index, sample in enumerate(samples):
                    for j in range(len(main_graph.nodes())):
                        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                        avars.append(a)
                        var_active_dict[(sample_index,j)] = a    

                mvars.append(avars)
                mvars.append(svars)

                obj_expr = quicksum(avars)
                model.setObjective(min_value, GRB.MAXIMIZE)

                model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)

                for sample_index, sample in enumerate(samples):
                    for i in range(len(main_graph.nodes())):
                        neighbors = sample.in_edges(i) 
                        e = len(neighbors)
                        ai = var_active_dict[(sample_index,i)]
                        si = var_seed_dict[i]
                        if i in  var_mean_dict.keys():
                            var_mean_dict[i]+= ai
                        else:
                            var_mean_dict[i]= ai
                        neighbors_vars = []
                        for neighbor in neighbors:
                            neighbors_vars.append(var_active_dict[(sample_index,neighbor[0])])
                        expr = (e+1)*ai-si-quicksum(neighbors_vars)
                        model.addConstr(expr, GRB.GREATER_EQUAL, 0)
                        model.addConstr(expr, GRB.LESS_EQUAL, e)

                mean_label_dict = {}
                for label in label_dict.keys():
                    for node in label_dict[label]:
                        if label in mean_label_dict.keys():
                            mean_label_dict[label]+=var_mean_dict[node]
                        else:
                            mean_label_dict[label]=var_mean_dict[node]
        
        
                for label in label_dict.keys():
                    label_size = len(label_dict[label])
                    model.addConstr(mean_label_dict[label], GRB.GREATER_EQUAL, label_size *m* min_value)

                try:
                    model.optimize()
                except e:
                    print(e)

                with open('../../../../Git/influence_maximization/experiments/im500/results/fairmip/mean/base100/'+attribute+'/output_'+str(index)+'.txt', "w") as of:    
                    for key,value in var_seed_dict.items():
                        if(value.x > 0):
                            print(key, file=of)

In [ ]:
MIP_IM()

Optimize a model with 100008 rows, 50501 columns and 298609 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64288 rows and 32207 columns
Presolve time: 1.69s
Presolved: 35720 rows, 18294 columns, 129837 nonzeros
Found heuristic solution: objective 0.0419424
Variable types: 1 continuous, 18293 integer (18293 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 35720 rows, 18294 columns, 129837 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   25622    1.8528904e-01   0.000000e+00   4.084801e-01      5s
Concurrent spin time: 1.32s

Solved with dual simplex

Root relaxation: objective 2.134801e-01, 24479 iterations, 6.00 seconds

    Nodes    | 

 58077 10976    0.20978   23  360    0.20895    0.20996  0.48%  28.4  585s
 59055 10979     cutoff   84         0.20895    0.20995  0.48%  28.4  591s
 59992 11015    0.20959   31  335    0.20895    0.20994  0.48%  28.3  596s
 61147 10988 infeasible   26         0.20895    0.20993  0.47%  28.3  601s
 61802 10902 infeasible  115         0.20895    0.20992  0.47%  28.3  605s
 62924 10841    0.20941   63  411    0.20895    0.20991  0.46%  28.3  611s
 64110 10725 infeasible  103         0.20895    0.20989  0.45%  28.2  616s
 64944 10701    0.20933   40  324    0.20895    0.20988  0.45%  28.2  620s
 66498 10579 infeasible   34         0.20895    0.20986  0.44%  28.2  626s
 67594 10482    0.20917   54  362    0.20895    0.20985  0.43%  28.2  631s
 68708 10389 infeasible  108         0.20895    0.20984  0.43%  28.1  635s
 69784 10239    0.20902   53  278    0.20895    0.20983  0.42%  28.1  640s
 70865 10112 infeasible   72         0.20895    0.20982  0.42%  28.1  645s
 72087 10079    0.20942  

  9670  6246    0.21371   31  378    0.21220    0.21536  1.49%  24.7  145s
 10462  6720    0.21362   73  402    0.21220    0.21531  1.46%  24.6  150s
 11741  7544    0.21318   84  257    0.21220    0.21526  1.44%  24.1  157s
 12759  8148    0.21272   53  247    0.21220    0.21522  1.42%  23.6  160s
 14250  9037    0.21331   45  327    0.21220    0.21516  1.39%  23.2  166s
 15740  9952    0.21301   41  325    0.21220    0.21511  1.37%  22.8  170s
 16810 10672    0.21410   34  367    0.21220    0.21508  1.36%  22.4  175s
 17880 11295    0.21407   37  408    0.21220    0.21506  1.34%  22.1  180s
 19291 12174    0.21282   94  204    0.21220    0.21503  1.33%  21.9  185s
 21036 13098    0.21446   26  266    0.21220    0.21497  1.30%  21.9  199s
 21540 13369    0.21334   37  320    0.21220    0.21496  1.30%  21.8  200s
 22928 14104    0.21303   43  251    0.21220    0.21493  1.28%  21.7  205s
 24665 15109 infeasible   56         0.21220    0.21490  1.27%  21.7  211s
 26257 16036    0.21313  

 150664 72717 infeasible   41         0.21220    0.21419  0.93%  19.9  695s
 152053 73221    0.21294   70  172    0.21220    0.21418  0.93%  19.9  700s
 153618 73826 infeasible   52         0.21220    0.21418  0.93%  19.9  705s
 155151 74520    0.21243   74  216    0.21220    0.21418  0.93%  19.8  710s
 156491 75098    0.21256   75  228    0.21220    0.21417  0.93%  19.8  715s
 157952 75626     cutoff   59         0.21220    0.21417  0.93%  19.8  720s
 159386 76226    0.21277   57  292    0.21220    0.21417  0.92%  19.8  725s
 160856 76872    0.21391   37  176    0.21220    0.21416  0.92%  19.8  730s
 162405 77490    0.21359   39  319    0.21220    0.21416  0.92%  19.8  735s
 163863 78066     cutoff   39         0.21220    0.21415  0.92%  19.8  740s
 165262 78562    0.21284   50  327    0.21220    0.21415  0.92%  19.8  745s
 166820 79264    0.21269   46  306    0.21220    0.21415  0.92%  19.8  750s
 168285 79887    0.21376   29  369    0.21220    0.21414  0.91%  19.8  755s
 169603 8042

 302827 130341    0.21248   54  176    0.21220    0.21392  0.81%  19.4 1235s
 304257 130871    0.21231   55  279    0.21220    0.21391  0.81%  19.4 1240s
 305632 131393    0.21283   56  343    0.21220    0.21391  0.80%  19.4 1245s
 307124 131924    0.21300   33  167    0.21220    0.21391  0.80%  19.4 1250s
 308964 132622    0.21289   38  192    0.21220    0.21391  0.80%  19.4 1256s
 310391 133104     cutoff   38         0.21220    0.21391  0.80%  19.4 1261s
 311289 133409    0.21356   33  352    0.21220    0.21391  0.80%  19.4 1265s
 312798 133999 infeasible   44         0.21220    0.21390  0.80%  19.4 1270s
 314295 134521     cutoff   54         0.21220    0.21390  0.80%  19.4 1276s
 315790 134919     cutoff   37         0.21220    0.21390  0.80%  19.4 1280s
 317191 135389    0.21334   58  191    0.21220    0.21390  0.80%  19.4 1285s
 318613 135890    0.21370   42  335    0.21220    0.21390  0.80%  19.4 1290s
 320060 136454    0.21327   67  233    0.21220    0.21389  0.80%  19.4 1295s

...

 607890 190659 infeasible   32         0.21252    0.21359  0.50%  19.3 2285s
 609714 190879    0.21303   52  234    0.21252    0.21358  0.50%  19.3 2291s
 610945 190998    0.21357   57  328    0.21252    0.21358  0.50%  19.3 2295s
 612732 191198    0.21327   39  236    0.21252    0.21358  0.50%  19.3 2301s
 613770 191353    0.21314   54  252    0.21252    0.21358  0.50%  19.3 2305s
 615129 191515    0.21311   40  362    0.21252    0.21358  0.50%  19.3 2310s
 616826 191636     cutoff   46         0.21252    0.21358  0.50%  19.3 2316s
 618120 191729 infeasible   46         0.21252    0.21358  0.50%  19.3 2321s
 619140 191795    0.21287   35  282    0.21252    0.21357  0.50%  19.3 2326s
 620355 191837 infeasible   73         0.21252    0.21357  0.50%  19.3 2331s
 621631 191942    0.21309   35  222    0.21252    0.21357  0.50%  19.3 2335s
 622941 192064 infeasible   69         0.21252    0.21357  0.49%  19.3 2340s
 624670 192230    0.21286   62  334    0.21252    0.21357  0.49%  19.3 2346s

 759031 199013     cutoff   79         0.21252    0.21345  0.44%  19.3 2820s
 760488 198996 infeasible   65         0.21252    0.21345  0.44%  19.3 2826s
 761747 198976     cutoff   81         0.21252    0.21345  0.44%  19.3 2831s
 763040 198998    0.21320   51  253    0.21252    0.21345  0.44%  19.3 2835s
 764620 198991     cutoff   38         0.21252    0.21345  0.44%  19.3 2841s
 765724 198952    0.21278   55  261    0.21252    0.21345  0.44%  19.3 2845s
 767394 198930    0.21309   33  281    0.21252    0.21344  0.43%  19.3 2850s
 768586 198947    0.21280   38  337    0.21252    0.21344  0.43%  19.3 2855s
 770309 199084    0.21265   32  153    0.21252    0.21344  0.43%  19.3 2861s
 771560 199114    0.21290   42  483    0.21252    0.21344  0.43%  19.3 2865s
 772771 199070    0.21289   46  379    0.21252    0.21344  0.43%  19.3 2870s
 774355 199058     cutoff   71         0.21252    0.21344  0.43%  19.3 2876s
 775983 199094 infeasible   41         0.21252    0.21344  0.43%  19.3 2881s

 905720 195459     cutoff   41         0.21252    0.21333  0.38%  19.2 6155s
 907158 195366 infeasible   65         0.21252    0.21333  0.38%  19.2 6160s
 908099 195268    0.21323   42  218    0.21252    0.21333  0.38%  19.2 6165s
 909642 195186     cutoff   47         0.21252    0.21333  0.38%  19.2 6171s
 911167 195094     cutoff   51         0.21252    0.21333  0.38%  19.2 6175s
 912648 195066 infeasible   45         0.21252    0.21333  0.38%  19.2 6180s
 914221 195032 infeasible   86         0.21252    0.21333  0.38%  19.2 6185s
 915695 194960     cutoff   52         0.21252    0.21332  0.38%  19.2 6190s
 917123 194853    0.21312   53  373    0.21252    0.21332  0.38%  19.2 6195s
 918442 194753 infeasible   45         0.21252    0.21332  0.38%  19.2 6200s
 920054 194717    0.21301   44  373    0.21252    0.21332  0.38%  19.2 6205s
 921812 194583 infeasible   59         0.21252    0.21332  0.38%  19.2 6211s
 923033 194489 infeasible   48         0.21252    0.21332  0.38%  19.2 6215s

 1063764 180570    0.21279   44  308    0.21252    0.21321  0.33%  19.0 6685s
 1065368 180396    0.21307   58  217    0.21252    0.21321  0.33%  19.0 6690s
 1066857 180154 infeasible   85         0.21252    0.21321  0.33%  19.0 6695s
 1068413 179925    0.21281   44  323    0.21252    0.21321  0.32%  19.0 6700s
 1069727 179774     cutoff   34         0.21252    0.21321  0.32%  19.0 6705s
 1071517 179450    0.21319   42  259    0.21252    0.21321  0.32%  19.0 6710s
 1073132 179252 infeasible   31         0.21252    0.21321  0.32%  19.0 6715s
 1074478 178972    0.21297   27  369    0.21252    0.21321  0.32%  19.0 6720s
 1076018 178792     cutoff   41         0.21252    0.21320  0.32%  19.0 6726s
 1076958 178567     cutoff   72         0.21252    0.21320  0.32%  19.0 6730s
 1078670 178351    0.21283   36  389    0.21252    0.21320  0.32%  19.0 6735s
 1080239 178143 infeasible   56         0.21252    0.21320  0.32%  19.0 6740s
 1081738 177935    0.21282   40  275    0.21252    0.21320  0.32

 1211233 153815     cutoff   43         0.21252    0.21310  0.27%  18.9 7215s
 1212602 153421     cutoff   64         0.21252    0.21310  0.27%  18.9 7220s
 1214031 153044     cutoff   55         0.21252    0.21310  0.27%  18.9 7225s
 1215382 152774     cutoff   44         0.21252    0.21310  0.27%  18.9 7230s
 1216900 152425    0.21291   45  157    0.21252    0.21310  0.27%  18.8 7235s
 1218252 152026 infeasible   49         0.21252    0.21310  0.27%  18.8 7240s
 1219491 151728 infeasible   69         0.21252    0.21310  0.27%  18.8 7245s
 1221050 151398     cutoff   58         0.21252    0.21309  0.27%  18.8 7250s
 1222512 151019 infeasible   39         0.21252    0.21309  0.27%  18.8 7255s
 1224109 150542    0.21289   42  285    0.21252    0.21309  0.27%  18.8 7260s
 1225613 150161 infeasible   63         0.21252    0.21309  0.27%  18.8 7265s
 1227029 149749 infeasible   58         0.21252    0.21309  0.27%  18.8 7270s
 1228401 149451 infeasible   53         0.21252    0.21309  0.27